In [1]:
import sys,os
sys.path.append('../../')
from smodels.base import runtime
# Define your model (list of BSM particles)
runtime.modelFile = 'smodels.share.models.mssm'
# runtime.modelFile = 'mssmQNumbers.slha'

from smodels.decomposition import decomposer
from smodels.base.physicsUnits import fb, GeV, TeV
from smodels.matching.theoryPrediction import theoryPredictionsFor,TheoryPredictionsCombiner
from smodels.experiment.databaseObj import Database
from smodels.base.smodelsLogging import setLogLevel
from smodels.share.models.mssm import BSMList
from smodels.share.models.SMparticles import SMList
from smodels.base.model import Model
setLogLevel("error")

# Set the path to the database
database = Database('unittest')

### Load model

In [2]:
slhafile = '../../inputFiles/slha/simplyGluino.slha'
model = Model(BSMparticles=BSMList, SMparticles=SMList)
model.updateParticles(inputFile=slhafile,
                              ignorePromptQNumbers=['spin','eCharge','colordim'])


### Decompose

In [3]:
# Set main options for decomposition
sigmacut = 0.005*fb
mingap = 5.*GeV

# Decompose model
topDict = decomposer.decompose(model, sigmacut,
                               massCompress=True, invisibleCompress=True,
                               minmassgap=mingap)

### Load the experimental results to be used.

In [4]:
database.selectExpResults(analysisIDs=['ATLAS-SUSY-2013-02','CMS-SUS-13-012'])

### Compute the theory predictions

In [5]:
print("\n Theory Predictions and Constraints:")
rmax = 0.
bestResult = None
allPredictions = theoryPredictionsFor(database, topDict, combinedResults=False)
predsDict = {}
for tp in allPredictions:
    anaID = tp.analysisId()
    if anaID not in predsDict:
        predsDict[anaID] = []
    predsDict[anaID].append(tp)


for anaID,predictions in predsDict.items():
    if not predictions:
        continue  # Skip if there are no constraints from this result
    print('\n %s ' % anaID)
    for theoryPrediction in predictions:
        dataset = theoryPrediction.dataset
        datasetID = theoryPrediction.dataId()
        txnames = sorted([str(txname) for txname in theoryPrediction.txnames])
        print("------------------------")
        print("Dataset = ", datasetID)  # Analysis name
        print("TxNames = ", txnames)
        print("Theory Prediction = ", theoryPrediction.xsection)  # Signal cross section
        print("Condition Violation = ", theoryPrediction.conditions)  # Condition violation values

        # Get the corresponding upper limit:
        print("UL for theory prediction = ", theoryPrediction.upperLimit)

        # Compute the r-value
        r = theoryPrediction.getRValue()
        print("r = %1.3E" % r)
        # Compute likelihoods for EM-type results:
        if dataset.getType() == 'efficiencyMap':
            theoryPrediction.computeStatistics()
            print('L_BSM, L_SM, L_max = %1.3E, %1.3E, %1.3E' % (theoryPrediction.likelihood(),
                  theoryPrediction.lsm(), theoryPrediction.lmax()))
        if r > rmax:
            rmax = r
            bestResult = anaID
        allPredictions.append(theoryPrediction)


 Theory Predictions and Constraints:

 ATLAS-SUSY-2013-02 
------------------------
Dataset =  None
TxNames =  ['T1']
Theory Prediction =  5.72E-01 [pb]
Condition Violation =  None
UL for theory prediction =  3.81E+01 [fb]
r = 1.500E+01

 CMS-SUS-13-012 
------------------------
Dataset =  6NJet8_1000HT1250_450MHTinf
TxNames =  ['T1']
Theory Prediction =  1.72E-03 [pb]
Condition Violation =  None
UL for theory prediction =  3.86E-01 [fb]
r = 4.450E+00
L_BSM, L_SM, L_max = 1.267E-11, 4.876E-02, 7.085E-02
